In [1]:
import pandas as pd
DATADIR = "D:/Documents/UFMG/PIC/RedesNeurais/kaggle_dataset/diabetic-retinopathy-classified/diabetic-retinopathy-customized/trainLabels.csv"

trainLabels = pd.read_csv(DATADIR)
trainLabels.head()


,image,level
0,10_left,0
1,10_right,0
2,13_left,0
3,13_right,0
4,15_left,1


In [2]:
from PIL import Image
from keras.preprocessing import image
import os
import numpy as np

# resize the image to (256, 256)
img_rows, img_cols = 128, 128

listing = os.listdir("D:/Documents/UFMG/PIC/RedesNeurais/kaggle_dataset/diabetic-retinopathy-classified/diabetic-retinopathy-customized/resized_train_cropped/resized_train_cropped/binary/dataset_dividido") 
'''listing.remove("trainLabels.csv")'''

immatrix = []
imlabel = []
classes = 2

'''listing
'''

Using TensorFlow backend.


'listing\n'

In [3]:

for file in listing:
    base = os.path.basename("D:/Documents/UFMG/PIC/RedesNeurais/kaggle_dataset/diabetic-retinopathy-classified/diabetic-retinopathy-customized/resized_train_cropped/resized_train_cropped/binary/dataset_dividido/" + file)
    fileName = os.path.splitext(base)[0]
    imlabel.append(trainLabels.loc[trainLabels.image==fileName, 'level'].values[0])
    im = Image.open("D:/Documents/UFMG/PIC/RedesNeurais/kaggle_dataset/diabetic-retinopathy-classified/diabetic-retinopathy-customized/resized_train_cropped/resized_train_cropped/binary/dataset_dividido/" + file)
    img = np.array(im.resize((img_rows,img_cols)))
    
    # convert to green channel only
    '''img[:,:,[0]] = 0'''
    immatrix.append(img)
    length = len(immatrix)
    if(length % 1000 == 0):
        print(length)


'''import matplotlib.pyplot as plt 
plt.imshow(img)
img'''

KeyboardInterrupt: 

In [ ]:
im = Image.fromarray(immatrix[1],'RGB')
print("level:",imlabel[1])
im

In [ ]:
import random

# define transformation methods
def horizontal_flip(image_array):
    return image_array[:, ::-1]

def vertical_flip(image_array):
    return image_array[::-1,:]

def aneritra_filter(image_array):
    image_array[:,:,[0]] = 0
    return image_array


def random_transform(image_array):
    if random.random() < 0.5:
        return vertical_flip(image_array)
    else:
        return horizontal_flip(image_array)

In [ ]:
im = Image.fromarray(vertical_flip(immatrix[1]),'RGB')
im

In [ ]:
length = len(immatrix)
for i in range(length):
    immatrix.append(horizontal_flip(immatrix[i]))
    imlabel.append(imlabel[i])
    immatrix.append(vertical_flip(immatrix[i]))
    imlabel.append(imlabel[i])
    immatrix.append(aneritra_filter(immatrix[i]))
    imlabel.append(imlabel[i])
    
        
print("Size of image array before augmentation: ", length)
print("Size fo image array after augmentation: ", len(immatrix))

In [ ]:
from sklearn.utils import shuffle

'''data,label = shuffle(immatrix, imlabel, random_state=42)'''
train_data = [immatrix,imlabel]

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train_data[0], train_data[1], test_size = 0.1, random_state = 42)

print(np.array(x_train).shape)
print(np.array(y_train).shape)

In [ ]:
from keras.utils import np_utils

y_train = np_utils.to_categorical(np.array(y_train), classes)
y_test = np_utils.to_categorical(np.array(y_test), classes)

x_train = np.array(x_train).astype("float32")/255.
x_test = np.array(x_test).astype("float32")/255.

print(np.array(y_train).shape)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

model = Sequential()

model.add(Conv2D(256, (3, 3), padding='same', activation='relu', input_shape=x_train[0].shape))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(activation='softmax', units=classes))

model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.adam(lr=0.001), metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss')]
model.fit(x_train, y_train,
          batch_size = 64,
          epochs=1,
          validation_split = 0.1,
          callbacks = callbacks)


In [ ]:
predictions = model.predict(x_test)
predictions

In [ ]:
score = model.evaluate(x_test, y_test, verbose=1)
print(score)